In [ ]:
import redis
import os
from langchain_aws.vectorstores.inmemorydb import InMemoryVectorStore

### We will be using the Titan Embeddings Model to generate our Embeddings.

In [ ]:
from langchain_aws.embeddings import BedrockEmbeddings
from langchain_aws import BedrockLLM

###  Define  the Anthropic Model params 

In [ ]:
# create the Anthropic Model
model_kwargs = {
    "max_tokens_to_sample": 8000,
    "temperature": 0, 
    "top_k": 250, 
    "top_p": 1,
    "stop_sequences": ["\\n\\nHuman:"]
}    

### Initialize large language model and use model properties for Claude-v2

In [ ]:
# use the Anthropic Claude model
llm = BedrockLLM(
    model_id="anthropic.claude-v2",
    model_kwargs=model_kwargs
)

### Define titan embeddings model

In [ ]:
# create a Titan Embeddings client
embeddings = BedrockEmbeddings()
  

##### Here is the document we load for using in context. 


In [ ]:
pdf_path = "./memorydb-guide.pdf"
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

[Go to Section Title](#section_name)


### Pre process the data to split into chunks that can be loaded into Vector database. 

In [ ]:
%%time
loader = PyPDFLoader(file_path=pdf_path) #load the pdf file
pages = loader.load_and_split()
# pages[10] # Uncomment if you want to see the data 

text_splitter = RecursiveCharacterTextSplitter( #create a text splitter
        separators=["\n\n", "\n", ".", " "], #split chunks at (1) paragraph, (2) line, (3) sentence, or (4) word, in that order
        chunk_size=1000, #divide into 1000-character chunks using the separators above
        chunk_overlap=100 #number of characters that can overlap with previous chunk
    )
chunks = loader.load_and_split(text_splitter)

## Using MemoryDB as Vector store. 
We test out both Semantic Search and using MemoryDB as retriever for RAG. 
We are using MemoryDB for our vector store. This code tests connection to MemDB and clears the existing data. 

**Comment client_devo.flushall() if you dont want to clear the data and index creation**


In [ ]:
from redis.cluster import RedisCluster as MemoryDBCluster

In [ ]:
endpoint = "<your-cluster-endpoint>"

rc = MemoryDBCluster(host=f"{endpoint}", 
           port=6379,ssl=True, decode_responses=True, ssl_cert_reqs="none")

rc.ping()
rc.flushall()

In [ ]:
INDEX_NAME='idx:vss-mm'

In [ ]:
vector_schema = {
    "algorithm": "HNSW"
}


### Create the index and Load the documents with their embeddings into Redis. 

In [ ]:
%%time
vds = InMemoryVectorStore.from_documents(
            chunks,
            embeddings,
            redis_url=f"rediss://{endpoint}:6379/ssl=True&ssl_cert_reqs=none",
            vector_schema=vector_schema,
            index_name=INDEX_NAME,
        )

### Lets inspect the index we created 

In [ ]:
%%time
info = rc.ft(INDEX_NAME).info()
num_docs = info['num_docs']
space_usage = info['space_usage']
num_indexed_vectors = info['num_indexed_vectors']
vector_space_usage = (info['vector_space_usage'])

print(f"{num_docs} documents ({space_usage} space used vectors indexed {num_indexed_vectors} vector space usage in {vector_space_usage}")

### Testing similarity search 

[Here are some search functions](https://python.langchain.com/docs/integrations/vectorstores/redis#querying)!

## Run this if the index is already created and data is loaded 

In [ ]:
%%time
vds = InMemoryVectorStore(
        redis_url=f"rediss://{endpoint}:6379/ssl=True&ssl_cert_reqs=none",
        index_name=INDEX_NAME,
        embedding=embeddings,
       # index_schema=index_schema,  # Replace with your index schema if needed
)


In [ ]:
query = "how to do backups with memoryDB?"


In [ ]:
%%time
results = vds.similarity_search(query)
(results)

In [ ]:
for item in results:
    print(item.page_content, end="\n\n")

## RAG 

### RAG
The below code helps implement MemoryDB vector database as a retriever. By default, it will use [**semantic similarity**](https://python.langchain.com/docs/integrations/vectorstores/redis#redis-as-retriever)!

[Retreival Augmented Generation](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html)!

We will use MemoryDB developer Guide 

In [ ]:
%%time
retriever = vds.as_retriever()

In [ ]:
%%time
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Use the following pieces of context to provide a concise answer in English to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa_prompt = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
    #verbose="true"
)
query = "How do i create a MemoryDB cluster?"
result = qa_prompt({"query": query})
print(result["result"])